In [1]:
import langchain

print(langchain.__version__)

1.1.1


In [2]:
!nvidia-smi

Sat Dec 13 22:55:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX150         WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   67C    P0            N/A  / 5001W |       0MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
from dotenv import load_dotenv
from typing import Annotated, Literal, TypedDict

from telethon import TelegramClient, errors
import asyncio
import re
import nltk
from nltk.corpus import stopwords
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Импорт общей конфигурации
import config as cfg

import torch

print(f"PyTorch версия: {torch.__version__}")
print(f"Версия CUDA в PyTorch: {torch.version.cuda}")
print("Доступность cuda", torch.cuda.is_available())

nltk.download("stopwords")
sw = stopwords.words("russian")

load_dotenv()

# Используем параметры из конфига
api_id = cfg.TELEGRAM_API_ID
api_hash = cfg.TELEGRAM_API_HASH
mistral_api_key = cfg.MISTRAL_API_KEY
channels = cfg.TELEGRAM_CHANNELS

d:\itmo\NewsHound\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch версия: 2.9.1+cpu
Версия CUDA в PyTorch: None
Доступность cuda False


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1 Подготовка данных

In [4]:
def clean_text_ru(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Замена всех не-словесных символов на пробел (кроме букв и знаков препинания)
    text = re.sub(r"\W+", " ", text)

    # Удаление URL-адресов
    text = re.sub(r"http\S+", "", text)

    # Создание шаблона для HTML-тегов
    html = re.compile(r"&lt;.*?&gt;")

    # Удаление HTML-тегов из текста
    text = html.sub(r"", text)

    # Список пунктуаций для удаления
    punctuations = "@#!?+&amp;*[]-%.:/();$=&gt;&lt;|{}^" + "'`" + "_"
    for p in punctuations:
        text = text.replace(p, "")  # Удаление пунктуации

    # Удаление стоп-слов и приведение слов к нижнему регистру
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    # Объединение слов обратно в текст
    text = " ".join(text)

    # Создание шаблона для поиска эмодзи
    emoji_pattern = re.compile(
        "["
        "\U0001f600-\U0001f64f"  # эмоции
        "\U0001f300-\U0001f5ff"  # символы и пиктограммы
        "\U0001f680-\U0001f6ff"  # транспорт и карты
        "\U0001f1e0-\U0001f1ff"  # флаги
        "\U00002702-\U000027b0"
        "\U000024c2-\U0001f251"
        "]+",
        flags=re.UNICODE,
    )

    # Удаление эмодзи из текста
    text = emoji_pattern.sub(r"", text)

    return text

In [5]:
client = TelegramClient("session_news", api_id, api_hash)


async def load_telegram_news(channel, limit=300):
    """
    Загружает последние сообщения из указанного Telegram-канала.
    """
    try:
        print(f"Подключение к Telegram для обработки канала {channel}...")
        await client.start()

        docs = []
        print(f"Загрузка сообщений из канала {channel}...")
        async for msg in client.iter_messages(channel, limit=limit):
            if not msg.text:
                continue

            # Очистка текста
            cleaned_text = clean_text_ru(msg.text)
            # print(f"Оригинальный текст: {msg.text[:50]}...")
            # print(f"Очищенный текст: {cleaned_text[:50]}...")

            docs.append(
                {
                    "id": msg.id,
                    "channel": channel,
                    "text": cleaned_text,
                    "date": msg.date.isoformat(),
                }
            )
        print(
            f"Загрузка из канала {channel} завершена. Получено {len(docs)} сообщений."
        )
        return docs
    except errors.UsernameInvalidError:
        print(f"Канал {channel} недоступен или не существует.")
        return []
    except errors.FloodWaitError as e:
        print(
            f"Превышен лимит запросов для канала {channel}. Ожидание {e.seconds} секунд."
        )
        await asyncio.sleep(e.seconds)
        return []
    except Exception as e:
        print(f"Ошибка при обработке канала {channel}: {e}")
        return []
    finally:
        print(f"Отключение от Telegram после обработки канала {channel}...")
        await client.disconnect()


def prepare_rag_documents(messages):
    """
    Преобразует сообщения в формат, подходящий для RAG.
    """
    print("Подготовка данных для RAG...")
    print(f"Количество сообщений для подготовки: {len(messages)}")  # Отладка
    rag_docs = [
        {
            "id": msg["id"],
            "content": msg["text"],
            "metadata": {
                "source": "telegram",
                "channel": msg["channel"],
                "date": msg["date"],
            },
        }
        for msg in messages
    ]
    print(f"Подготовлено {len(rag_docs)} документов.")
    return rag_docs


async def main():
    all_messages = []
    for channel in channels:
        print(f"Обработка канала {channel}...")
        messages = await load_telegram_news(channel, limit=100)
        print(f"Сообщений из канала {channel}: {len(messages)}")  # Отладка
        all_messages.extend(messages)

    if all_messages:
        print("Подготовка новостей для RAG...")
        rag_docs = prepare_rag_documents(all_messages)
        print("Пример подготовленного документа:")
        print(rag_docs[0] if rag_docs else "Нет документов для отображения.")  # Отладка
        return all_messages, rag_docs
    else:
        print("Нет доступных сообщений.")
        return [], []


# Запуск main() и получение данных
all_messages, rag_docs = await main()
print(f"Всего сообщений: {len(all_messages)}")  # Отладка
print(f"Всего документов для RAG: {len(rag_docs)}")  # Отладка

Обработка канала @tass_agency...
Подключение к Telegram для обработки канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка из канала @tass_agency завершена. Получено 95 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 95
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка из канала @rian_ru завершена. Получено 91 сообщений.
Отключение от Telegram после обработки канала @rian_ru...
Сообщений из канала @rian_ru: 91
Обработка канала @kommersant...
Подключение к Telegram для обработки канала @kommersant...
Загрузка сообщений из канала @kommersant...
Загрузка из канала @kommersant завершена. Получено 88 сообщений.
Отключение от Telegram после обработки канала @kommersant...
Сообщений из канала @kommersant: 88
Обработка канала @gazeta_ru...
Подключение к Telegram для обработки канала @gazeta_ru...
Загрузка сообщений из канала @ga

In [6]:
rag_docs[:3]

[{'id': 352828,
  'content': 'орбан заявил использование евросоюзом замороженных российских активов сути объявлением войны изъятие сотен миллиардов государства истории оставалось ответа',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-13T19:42:56+00:00'}},
 {'id': 352827,
  'content': 'ответ очень серьезным вашингтон ответит иг запрещена рф убийство двух военных сша одного американского переводчика пальмире заявил трамп',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-13T19:20:41+00:00'}},
 {'id': 352826,
  'content': 'белоусов вручил звезду героя россии оператору fv дрона центра рубикон позывным филин который уничтожил пять танков 169 автомобилей всу сообщили мо рф видео минобороны россии тасс',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-13T17:31:21+00:00'}}]

### 2 Подготовка поисквого индекса

In [7]:
# Используем параметры из конфига
splitter = RecursiveCharacterTextSplitter(
    chunk_size=cfg.CHUNK_SIZE,
    chunk_overlap=cfg.CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", "! ", "? ", ", ", " ", ""],
)
print(f"Chunk size: {cfg.CHUNK_SIZE}, overlap: {cfg.CHUNK_OVERLAP}")


def to_langchain_docs(raw_list):
    docs = []
    for item in raw_list:
        docs.append(Document(page_content=item["content"], metadata=item["metadata"]))
    return docs


def split_docs(rag_docs):
    chunks = splitter.split_documents(rag_docs)
    return chunks

Chunk size: 500, overlap: 50


In [8]:
docs = to_langchain_docs(rag_docs)
chunks = split_docs(docs)

In [9]:
chunks[:3]

[Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-13T19:42:56+00:00'}, page_content='орбан заявил использование евросоюзом замороженных российских активов сути объявлением войны изъятие сотен миллиардов государства истории оставалось ответа'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-13T19:20:41+00:00'}, page_content='ответ очень серьезным вашингтон ответит иг запрещена рф убийство двух военных сша одного американского переводчика пальмире заявил трамп'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-13T17:31:21+00:00'}, page_content='белоусов вручил звезду героя россии оператору fv дрона центра рубикон позывным филин который уничтожил пять танков 169 автомобилей всу сообщили мо рф видео минобороны россии тасс')]

### 3 Embedder

### 4 Выбор векторного хранилища: FAISS (in-memory) или Qdrant (Docker)

**Qdrant** — для production (данные сохраняются в Docker контейнере)
```bash
docker-compose up -d
```

**FAISS** — для быстрых экспериментов (данные в памяти)

In [10]:
# === ВАРИАНТ A: Qdrant (Docker) ===
# Раскомментируйте этот блок для использования Qdrant

from vectorstore import VectorStore

# Подключение к Qdrant (параметры из config.py)
qdrant_store = VectorStore()

# Создание коллекции и загрузка документов
# recreate=True пересоздаст коллекцию (удалит старые данные)
qdrant_store.create_collection(recreate=True)
qdrant_store.add_documents(rag_docs)

# Проверка
print(qdrant_store.get_collection_info())


# Создаем retriever-совместимый интерфейс
class QdrantRetriever:
    def __init__(self, store, k=None):
        self.store = store
        self.k = k or cfg.RETRIEVAL_K

    def invoke(self, query):
        results = self.store.search(query, k=self.k)
        # Преобразуем в формат LangChain Document
        return [
            Document(page_content=r["content"], metadata=r["metadata"]) for r in results
        ]


retriever = QdrantRetriever(qdrant_store)
vectorstore = qdrant_store  # для совместимости

print(f"✅ Qdrant vectorstore готов! (k={cfg.RETRIEVAL_K})")

Подключение к Qdrant: localhost:6333


d:\itmo\NewsHound\rag\vectorstore.py:30: UserWarning: Qdrant client version 1.16.2 is incompatible with server version 1.14.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self.client = QdrantClient(host=self.host, port=self.port)
d:\itmo\NewsHound\rag\vectorstore.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Подключено к Qdrant
Загрузка модели эмбеддингов: sergeyzh/rubert-mini-frida


Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


Размер вектора: 312
Создание коллекции: news
Коллекция создана
Коллекция news уже существует
Добавление 442 документов...
  Загружено 100/442 документов
  Загружено 200/442 документов
  Загружено 300/442 документов
  Загружено 400/442 документов
Загружено 442 документов
{'error': "'CollectionInfo' object has no attribute 'vectors_count'"}
✅ Qdrant vectorstore готов! (k=5)


In [ ]:
# === ВАРИАНТ B: FAISS (in-memory) ===
# Раскомментируйте этот блок для использования FAISS

# vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
# retriever = vectorstore.as_retriever(search_kwargs={"k": cfg.RETRIEVAL_K})
# print(f"✅ FAISS vectorstore готов! (k={cfg.RETRIEVAL_K})")

In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Используем модель из конфига
embeddings = HuggingFaceEmbeddings(
    model_name=cfg.EMBEDDING_MODEL,
    model_kwargs={"device": cfg.EMBEDDING_DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)
print(f"Модель эмбеддингов: {cfg.EMBEDDING_MODEL}")

Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


Модель эмбеддингов: sergeyzh/rubert-mini-frida


### 4 Подготовка поискового движка

In [22]:
# FAISS вариант (закомментирован - используйте ячейки 13-14 для выбора Qdrant или FAISS)
# vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

### 5 Построение Retrieval-Augmented Generation (RAG)

In [23]:
from langchain_mistralai import ChatMistralAI

# Используем модель из конфига
llm = ChatMistralAI(model=cfg.LLM_MODEL, max_retries=2, api_key=cfg.MISTRAL_API_KEY)
llm.verbose = False
print(f"LLM модель: {cfg.LLM_MODEL}")

LLM модель: mistral-small-latest


In [24]:
llm.invoke("Hi!")

AIMessage(content='Hello! 😊 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 5, 'total_tokens': 18, 'completion_tokens': 13}, 'model_name': 'mistral-small-latest', 'model': 'mistral-small-latest', 'finish_reason': 'stop'}, id='lc_run--6dbe9ce9-f8fd-4866-bb7d-f0b7eadcc5a5-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18})

In [25]:
import requests
import os

api_key = os.getenv("MISTRAL_API_KEY")
url = "https://api.mistral.ai/v1/models"
headers = {"Authorization": f"Bearer {api_key}"}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    models = response.json()["data"]
    print("Доступные модели Mistral:")
    for model in models:
        print(f"- {model['id']}")
else:
    print(f"Ошибка: {response.status_code} - {response.text}")

Доступные модели Mistral:
- mistral-medium-2505
- mistral-medium-2508
- mistral-medium-latest
- mistral-medium
- open-mistral-7b
- mistral-tiny
- mistral-tiny-2312
- open-mistral-nemo
- open-mistral-nemo-2407
- mistral-tiny-2407
- mistral-tiny-latest
- mistral-large-2411
- pixtral-large-2411
- pixtral-large-latest
- mistral-large-pixtral-2411
- codestral-2508
- codestral-latest
- devstral-small-2507
- devstral-medium-2507
- mistral-vibe-cli-latest
- devstral-2512
- devstral-medium-latest
- labs-devstral-small-2512
- devstral-small-latest
- mistral-small-2506
- mistral-small-latest
- magistral-medium-2509
- magistral-medium-latest
- magistral-small-2509
- magistral-small-latest
- voxtral-mini-2507
- voxtral-mini-latest
- voxtral-small-2507
- voxtral-small-latest
- mistral-large-2512
- mistral-large-latest
- ministral-3b-2512
- ministral-3b-latest
- ministral-8b-2512
- ministral-8b-latest
- ministral-14b-2512
- ministral-14b-latest
- pixtral-12b-2409
- pixtral-12b
- pixtral-12b-latest
- 

In [26]:
# Создаем ретривер на основе векторного хранилища
# Если используется Qdrant (ячейка 13), retriever уже создан
# Если используется FAISS, раскомментируйте строку ниже:
# retriever = vectorstore.as_retriever(search_kwargs={"k": cfg.RETRIEVAL_K})

In [27]:
# --- 1. Создаем цепочку для генерации поискового запроса (Query Expansion) ---
# Эта часть отвечает за то, чтобы превратить "как там нефть" в "последние новости о добыче нефти, ценах и влиянии на рынок"

query_gen_system_prompt = """Ты — помощник, который оптимизирует вопросы для поиска в базе данных новостей. 
Твоя задача — переформулировать вопрос пользователя так, чтобы он стал более точным, полным и подходящим для семантического поиска. 
Убери разговорные обороты. Добавь ключевые слова, если вопрос слишком общий. В вопросе не указывай года или даты.
Выведи ТОЛЬКО текст улучшенного поискового запроса, без кавычек и пояснений."""

query_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", query_gen_system_prompt), ("human", "Вопрос пользователя: {input}")]
)

# Создаем цепочку: Промпт -> LLM -> Парсер строки
query_gen_chain = query_gen_prompt | llm | StrOutputParser()


# --- 2. Создаем цепочку для ответов с Системным промптом (System Prompt) ---
# Эта часть отвечает за формирование финального ответа на основе найденных документов

system_prompt_template = """Ты — профессиональный новостной аналитик. Твоя задача — отвечать на вопросы, основываясь ИСКЛЮЧИТЕЛЬНО на предоставленном контексте новостей.

Правила:
1. Используй только информацию из контекста ниже. Не добавляй факты из своей памяти, если их нет в новостях.
2. Если в контексте нет информации для ответа, честно скажи: "К сожалению, в доступных новостях нет информации по вашему запросу".
3. Ответ должен быть структурированным, объективным и на русском языке.
4. Если новостей несколько, попытайся обобщить их.

Контекст новостей:
{context}"""

answer_prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt_template), ("human", "{input}")]
)

# Создаем цепочку обработки документов
document_chain = create_stuff_documents_chain(llm, answer_prompt)


# --- 3. Собираем всё вместе в единую функцию ---


def process_rag_query(user_question):
    print(f"🔵 Исходный запрос: {user_question}")

    # Шаг 1: Query expansion - расширение запроса
    search_query = query_gen_chain.invoke({"input": user_question})
    print(f"🟡 Сгенерированный поисковый запрос для FAISS: {search_query}")

    # Шаг 2: Запускаем ретривер для поиска релевантных документов
    retrieved_docs = retriever.invoke(search_query)
    print(f"🟢 Найдено документов: {len(retrieved_docs)}")

    # Шаг 3: Генерируем ответ, используя ОРИГИНАЛЬНЫЙ вопрос и найденные документы
    # Мы передаем 'input' (вопрос пользователя) и 'context' (документы) в document_chain
    response = document_chain.invoke(
        {"input": user_question, "context": retrieved_docs}
    )

    return response, retrieved_docs


# # --- Тестирование ---
# user_input = "как там новости про нефть"
# answer, sources = process_rag_query(user_input)

# print("\n" + "="*50 + "\n")
# print("📝 ОТВЕТ АНАЛИТИКА:\n")
# print(answer)
# print("\n" + "="*50 + "\n")
# print("📚 ИСТОЧНИКИ:")
# for i, doc in enumerate(sources):
#     print(f" {i}- [{doc.metadata.get('channel', 'Unknown')}]: {doc.page_content[:150]}...")

### 6 Постройка агента

Агент позволит использовать поиск в новостях как отдельный tool

In [28]:
# --- 1. Обернем наш RAG в инструмент (Tool) ---
@tool
def news_rag_tool(query: str):
    """
    Инструмент для поиска актуальных новостей и аналитики в базе знаний.
    Используй этот инструмент, когда пользователь спрашивает о событиях, новостях, политике.
    Входные данные: поисковый запрос (строка).
    """
    # Используем функцию process_rag_query, которую мы создали ранее
    # Она возвращает (ответ, документы), нам нужен только текст ответа для агента
    answer, _ = process_rag_query(query)
    return answer


tools = [news_rag_tool]

# --- 2. Привязываем инструменты к LLM ---
# Mistral поддерживает function calling, поэтому bind_tools сработает
agent_model = llm.bind_tools(tools)

In [29]:
# --- 3. Определяем состояние агента ---
class AgentState(TypedDict):
    # add_messages позволяет накапливать историю сообщений (append), а не перезаписывать её
    messages: Annotated[list, add_messages]


# --- 4. Определяем узлы (Nodes) ---


def agent_node(state: AgentState):
    """Узел, который вызывает LLM для принятия решения"""
    messages = state["messages"]
    response = agent_model.invoke(messages)
    return {"messages": [response]}


# Узел для выполнения инструментов (готовый компонент из langgraph)
tool_node = ToolNode(tools)


# --- 5. Определяем логику переходов (Edges) ---
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """Определяет, нужно ли вызывать инструменты или завершить работу"""
    messages = state["messages"]
    last_message = messages[-1]

    # Если LLM сгенерировала вызов инструмента (tool_calls)
    if last_message.tool_calls:
        return "tools"
    # Если LLM сгенерировала обычный текстовый ответ
    return "__end__"

In [30]:
# --- 6. Обновление Агента с проверкой на Off-topic ---

# 1. Создаем Системный промпт для Агента
# Здесь мы задаем личность агента и правила поведения
agent_system_prompt = """Ты — умный новостной ассистент NewsHound. 
Твоя главная цель — помогать пользователям находить информацию в базе новостей, используя инструмент 'news_rag_tool'.

Твои правила:
1. Если вопрос касается новостей, политики, экономики, финансов, событий в мире или конкретных компаний — СРАЗУ вызывай инструмент 'news_rag_tool'. Не пытайся отвечать сам.
2. Если вопрос НЕ касается новостей — ВЕЖЛИВО ОТКАЖИСЬ отвечать. Не предлагай улучшить запрос, или переформулировать.
3. Не вызывай инструмент для off-topic вопросов.
"""


# 2. Обновляем узел агента, чтобы он включал системный промпт
def agent_node_with_guardrail(state: AgentState):
    """Узел агента с системным промптом"""
    messages = state["messages"]

    # Добавляем системное сообщение в начало истории, если его там нет
    # (простая проверка, чтобы не дублировать при рекурсии, хотя LangGraph обычно обрабатывает это через state management,
    # но для надежности лучше явно сформировать список для вызова модели)

    if isinstance(messages[0], SystemMessage):
        # Если системное сообщение уже есть (например, от предыдущих шагов), оставляем как есть
        messages_to_send = messages
    else:
        # Если нет, добавляем наше
        messages_to_send = [SystemMessage(content=agent_system_prompt)] + messages

    response = agent_model.invoke(messages_to_send)
    return {"messages": [response]}


# 3. Пересобираем граф с новым узлом
workflow_v2 = StateGraph(AgentState)

# Добавляем узлы (используем новую функцию agent_node_with_guardrail)
workflow_v2.add_node("agent", agent_node_with_guardrail)
workflow_v2.add_node("tools", tool_node)

# Добавляем ребра
workflow_v2.add_edge(START, "agent")

# Условное ребро
workflow_v2.add_conditional_edges(
    "agent",
    should_continue,
)

# Возврат от инструментов к агенту
workflow_v2.add_edge("tools", "agent")

# Компилируем
app_v2 = workflow_v2.compile()

print("Агент v2 (с фильтрацией off-topic) успешно собран!")

Агент v2 (с фильтрацией off-topic) успешно собран!


In [31]:
# --- Тестирование Агента v2 ---


def test_agent(query):
    print(f"\n❓ Вопрос: {query}")
    print("-" * 20)
    inputs = {"messages": [HumanMessage(content=query)]}

    for output in app_v2.stream(inputs):
        for key, value in output.items():
            if key == "agent":
                msg = value["messages"][0]
                if msg.tool_calls:
                    print(
                        f"🛠️ Агент решил вызвать инструмент: {msg.tool_calls[0]['name']}"
                    )
                else:
                    print(f"🗣️ Агент решил ответить сам.")

    final_state = app_v2.invoke(inputs)
    print("-" * 20)
    print(f"🏁 Ответ: {final_state['messages'][-1].content}\n")


# Тест 1: По теме (должен вызвать tool)
test_agent("Какие прогнозы по инфляции?")

# Тест 2: Не по теме (должен отказать)
test_agent("Напиши рецепт борща")

# Тест 3: Не по теме (должен отказать)
test_agent("Напиши код на Python для сортировки списка")


❓ Вопрос: Какие прогнозы по инфляции?
--------------------
🛠️ Агент решил вызвать инструмент: news_rag_tool
🔵 Исходный запрос: Прогнозы по инфляции 2024
🟡 Сгенерированный поисковый запрос для FAISS: Прогнозы инфляции в России


AttributeError: 'QdrantClient' object has no attribute 'search'